# Lab 05. Regression and Clustering


In this lab we will tackle two types of tasks: 
- Regression Competition
- Clustering practice


#### Evaluation

Each task has its value, **15 points** in total. If you use some open-source code please make sure to include the url.

#### How to submit
- Name your file according to this convention: `2022_lab05_GroupNumber_Surname_Name.ipynb`, for example 
    - `2022_lab05_404_Sheipak_Sviat.ipynb`
    - `2022_lab05_M106_Sheipak_Sviat.ipynb`
- Attach your .ipynb to an email with topic `2022_lab05_GroupNumber_Surname_Name`
- Send it to `cosmic.research.ml@yandex.ru`
- Deadline is `2022-12-08 23:00:00 +03:00`

#### The Data:
- All the datasets you need are here: https://disk.yandex.ru/d/gqo8GmBMUBfRuw

## Part 1. Regression [7 points]

The task is to predict a price of a house sold in California based on some description of a house. Some columns give some information on the house itself (number of bedrooms, short written summary and so on) and some describe the neighborhood (middleschoolscore, middleschooldistance).

* Id column - `id`
* Target column - `sold_price`
* Scoring is `RMSE` - root mean squared error

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_train = pd.read_csv("housing_train.csv")
df_test = pd.read_csv("housing_test.csv")

In [ ]:
numeric_cols = ['bathrooms', 'full_bathrooms', 'total_interior_livable_area', 'total_spaces', 'garage_spaces', 
                'elementary_school_score', 'elementary_school_distance', 'middle_school_score', 'middle_school_distance', 
                'high_school_score', 'high_school_distance', 'tax_assessed_value', 'listed_price', 
                'last_sold_price', 'year_built', 'annual_tax_amount']

cat_cols = ['type', 'heating', 'cooling', 'parking', 'bedrooms', 'region',
            'elementary_school', 'middle_school', 'high_school', 'flooring', 
            'heating_features', 'cooling_features', 'appliances_included', 
            'laundry_features', 'parking_features', 'city', 'zip', 'state', 'listed_on', 'last_sold_on']

text_cols = ['address', 'summary',]
target_cols = ['sold_price']
id_cols = ['id']

In [ ]:
train_num_df = df_train[numeric_cols].fillna(df_train[numeric_cols].mean(axis=0))
test_num_df = df_test[numeric_cols].fillna(df_test[numeric_cols].mean(axis=0))

In [ ]:
X_train, Y_train = train_num_df.values, df_train[target_cols].values
X_test, Y_test = test_num_df.values, df_test[target_cols].values

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error

In [ ]:
lr_grid = {
    "alpha": np.logspace(-5, 3, 100)
}

lr_grid_searcher = GridSearchCV(Ridge(), lr_grid, cv=5, return_train_score=True, scoring="neg_root_mean_squared_error")
lr_grid_searcher.fit(X_train, Y_train)
lr_grid_searcher.best_estimator_,lr_grid_searcher.best_score_

In [ ]:
best_model = lr_grid_searcher.best_estimator_
best_model.fit(X_train, Y_train)

test_preds = best_model.predict(X_test)
np.sqrt(mean_squared_error(Y_test, test_preds))

Get a score as low as possible:

Table ref:
```
Score < 0.25 - 1 points
Score < 0.24 - 2 points
Score < 0.22 - 3 points
Score < 0.21 - 5 points
Score < 0.20 - 6 points
Score < 0.18 - 7 points
```

Don't forget to use categorical and text features.

**Task 1.1** [7 points]

In [ ]:
# YOUR CODE HERE

## Part 2. Clustering [8 points]

In this part we will try to analyze a dump of leaked passwords of internet users. It can be accessed here: https://github.com/ignis-sec/Pwdb-Public/tree/master/wordlists

First kind reminder - if you see your password in this base, change it immediately.

In [ ]:
words = []
with open("ignis-1M.txt", "r") as file:
    for line in file:
        words.append(line.strip())

To make it more simple, we'll use only first 3K of passwords:

In [ ]:
words = np.array(words[:3000]).reshape((-1, 1))

**Task 2.1 [0.5 point]**

Let's start with calculating levenshtein distance between words in the dataset. Compute a `3000x3000` distance matrix.

In [ ]:
import numpy as np
from pylev import levenshtein
import matplotlib.pyplot as plt

In [ ]:
X = np.zeros((words.shape[0], words.shape[0]))

# YOUR CODE HERE

In [ ]:
plt.hist(X.reshape(-1), rwidth=0.5)
plt.xticks(np.arange(0, X.max() + 1))
plt.show()

In [ ]:
plt.imshow(X, cmap="Purples")
plt.show()

**Task 2.2 [1.5 point]** First algorithm we'll use is `DBSCAN`.

We have to adjust two parameters:
- `eps`
- `min_samples`

Grid-search these two parameters and report number and sizes of output clusters for every pair of parameters. 

**Note**: to define an appropriate space for each parameter remember what they mean and how they affect DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

Example:

In [ ]:
eps = 3.0
min_samples = 3

db = DBSCAN(metric="precomputed", min_samples=min_samples, eps=eps).fit(X)
labels = db.labels_
len(set(labels))

In [ ]:
# YOUR CODE HERE

**Task 2.3 [1 point]** Choose a set of parameters that leads to 20-25 clusters.

- Is there a cluster that is significantly larger than the others? 
- How would you describe these clusters, what kind of passwords they contain? 

Use small samples from each cluster and try to describe a relevant password pattern.

In [ ]:
# YOUR CODE HERE

**Task 2.4 [1 point]** 

Let's try to improve clustering by introducing a custome levenshtein distance. You might have noticed that there are some specific password generation patterns, like `qwerty -> qwerty123`.

Classic levenshtein distance for these two passwords is 3. Try to define a custom levenshtein distance that would make these passwords closer.

Feel free to experiment and create as complex levenshtein distance as you would like.

Report new clustering, describe new clusters.


In [ ]:
#!pip3 install -U strsimpy

Example:

In [ ]:
from strsimpy.weighted_levenshtein import WeightedLevenshtein


def insertion_cost(char):
    return 1.0


def deletion_cost(char):
    return 1.0


def substitution_cost(char_a, char_b):
    if (char_a, char_b) == ('t', 'r') or (char_a, char_b) == ('r', 't'):
        return 0.5
    return 1.0

weighted_levenshtein = WeightedLevenshtein(
    substitution_cost_fn=substitution_cost,
    insertion_cost_fn=insertion_cost,
    deletion_cost_fn=deletion_cost)

In [ ]:
print(levenshtein('Stting1', 'String1'))
print(weighted_levenshtein.distance('Stting1', 'String1'))

In [ ]:
# YOUR CODE HERE

**Task 2.5 [1 point]** Hierarchical Agglomerative clustering

It is time to draw some pictures. 
- apply agglomerative clustering algorithm to form 5-10 clusters
- plot a dendrogram
- describe output clusters

In [ ]:
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist
from sklearn.cluster import AgglomerativeClustering

from collections import Counter

Compute dendrogram:

In [ ]:
condensed_X = # YOUR CODE HERE
linkage = hierarchy.linkage(# YOUR CODE HERE)

Plot it:

In [ ]:
# YOUR CODE HERE

Compute clusters:

In [ ]:
cluster = AgglomerativeClustering(# YOUR CODE HERE)
labels = cluster.predict(# YOUR CODE HERE)

Describe them in any form:

### K-Means
This clustering algorithm doesn't work with precomputed distances, as it has to calculate centroids and measure distance from a centroid to every object.

Thus, we need to map the dataset to some vector space. How? Embeddings of course

In [ ]:
import gensim.downloader

In [ ]:
list(gensim.downloader.info()['models'].keys())

In [ ]:
word_embeddings = gensim.downloader.load("glove-wiki-gigaword-100")

**Task 2.6 [1 point]** 

- Create two lists - for those passwords that can be embedded and for their embeddings correspondigly
- How many passwords have embeddings? 
- Describe the passwords that have embeddings and those that don't. Give your reasoning why these groups are formed like this.


In [ ]:
words_w_embeddings = [#YOUR CODE HERE]
embeddings = [#YOUR CODE HERE]

assert len(words_w_embeddings) = len(embeddings)

**Task 2.7 [2 point]** K-MEANS

- Run kmeans with different parameters, for every set of parameters report average in-class, out-class distance
- Remember that k-means has stochasticity, thus two algorithms with same hyperparameters can give different results
- Chose several (3-5) your favorite k-means versions, visualize clusters in 2D using PCA or TSNE
- Describe what are the clusters that kmeans can detect
- Are they different from DBSCAN? Why?

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

Example:

In [ ]:
embeddings_clusters = KMeans(n_clusters=3).fit_predict(embeddings)
    

pca = PCA(n_components=2)
pca_words = pca.fit_transform(embeddings)

plt.scatter(pca_words[:, 0], pca_words[:, 1])
plt.title("UNCOLOURED passwords PCA") # YOU HAVE TO PLOT IT WITH COLORS
plt.show()

**Task 2.8 [extra points]**

Here are some ideas how to experiment:
- compare performance of algorithms with levenshtein distance and embeddings
- use algorithms that were mentioned in the lecture, but with no explanation. In this section write a brief description of an algorithm before applying it

In [ ]:
# YOUR CODE HERE